In [1]:
import csv
import os

In [2]:
from PIL import Image
from PIL.ExifTags import TAGS

In [3]:
def get_exif(fn):
    ret = {}
    i = Image.open(fn)
    info = i._getexif()
    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        ret[decoded] = value
    return ret

In [4]:
def dmsToDD(d,m,s):
    ret = 0.0
    dm = float(m)+float(s/60.0)
    dd = float(d)+float(dm/60)
    return dd

In [5]:
flist = os.listdir()

In [6]:
images = []

In [7]:
#output CSV filename; You can change the filename in '' here!
OUTFILE = 'output.csv'

In [8]:
#populate images[] with file paths
for file in flist:
    if file.endswith('.jpg') or file.endswith('.JPG'):
        fpath = os.path.abspath(file)
        images.append(fpath)

In [9]:
outputFile = open(OUTFILE, 'w', newline='')

In [10]:
wrtr = csv.writer(outputFile)

In [11]:
wrtr.writerow(['FileName','Latitude_DMS','Longitude_DMS','Latitude_DD','Longitude_DD','Altitude_m','Altitude_ft'])

85

In [12]:
for image in images:
    imInfo = get_exif(image)
    fname = os.path.basename(image)
    #retrieve GPSInfo from photo metadata; dictionary with sub-tuples 
    gpsInfo = imInfo['GPSInfo']
    #retrieve latitude DMS from GPSInfo from photo
    latDeg = float(gpsInfo[2][0])
    latMin = float(gpsInfo[2][1])
    latMinInt = int(latMin)
    latSec = float(gpsInfo[2][2])
    latDD = dmsToDD(latDeg,latMin,latSec)
    #make lat degrees negative if in S hemisphere (S of equator)
    if gpsInfo[1] == 'S':
        latDeg = latDeg*-1.0
        latDD = latDD*-1.0
    latDegInt = int(latDeg)
    #retrieve longitude DMS from GPSInfo from photo
    lonDeg = float(gpsInfo[4][0])
    lonMin = float(gpsInfo[4][1])
    lonMinInt = int(lonMin)
    lonSec = float(gpsInfo[4][2])
    lonDD = dmsToDD(lonDeg,lonMin,lonSec)
    #make lon degrees negative is W of prime meridian
    if gpsInfo[3] == 'W':
        lonDeg = lonDeg*-1.0
        lonDD = lonDD*-1.0
    lonDegInt = int(lonDeg)
    latDMS = str(latDegInt)+' '+str(latMinInt)+' '+str(latSec)
    lonDMS = str(lonDegInt)+' '+str(lonMinInt)+' '+str(lonSec)
    #retrieve altitude in meters
    altM = float(gpsInfo[6])
    altF = altM*3.28084
    wrtr.writerow([str(fname),latDMS,lonDMS,str(latDD),str(lonDD),str(altM),str(altF)])

In [13]:
outputFile.close()

In [14]:
print('All finished! Output written to '+OUTFILE)

All finished! Output written to output.csv
